# Session 2.1: Introduction to DuckDB
Bootcamp Data Analysis - Python, DuckDB & Streamlit

## Topics:
1. What is DuckDB?
2. Setup and first connection
3. Basic SQL operations
4. Loading data

# Introduction to DuckDB

DuckDB adalah embedded analytical database yang:
- Fast untuk analytical queries
- Zero-dependency (no server needed)
- Support standard SQL
- Easy integration dengan Python/Pandas

In [1]:
import duckdb
import pandas as pd
import numpy as np
from pathlib import Path

print("DuckDB version:", duckdb.__version__)

DuckDB version: 1.4.1


## 1. Create Connection

In [2]:
# In-memory database
conn = duckdb.connect(':memory:')
print("✅ Connected to DuckDB (in-memory)")

✅ Connected to DuckDB (in-memory)


In [ ]:
# Or persistent database
# conn = duckdb.connect('my_database.db')

## 2. Basic SQL Operations

In [3]:
# Create table
conn.execute("""
    CREATE TABLE employees (
        id INTEGER,
        name VARCHAR,
        department VARCHAR,
        salary INTEGER
    )
""")

print("✅ Table 'employees' created")

✅ Table 'employees' created


In [4]:
# Insert data
conn.execute("""
    INSERT INTO employees VALUES
        (1, 'Alice', 'Engineering', 75000),
        (2, 'Bob', 'Sales', 65000),
        (3, 'Charlie', 'Engineering', 80000),
        (4, 'David', 'Marketing', 60000),
        (5, 'Eve', 'Engineering', 85000)
""")

print("✅ Data inserted")

✅ Data inserted


In [5]:
# Simple SELECT
result = conn.execute("SELECT * FROM employees").df()
print("\n📊 All Employees:")
print(result)


📊 All Employees:
   id     name   department  salary
0   1    Alice  Engineering   75000
1   2      Bob        Sales   65000
2   3  Charlie  Engineering   80000
3   4    David    Marketing   60000
4   5      Eve  Engineering   85000


In [6]:
# WHERE clause
result = conn.execute("""
    SELECT * FROM employees
    WHERE department = 'Engineering'
""").df()
print("\n👨‍💻 Engineering Department:")
print(result)


👨‍💻 Engineering Department:
   id     name   department  salary
0   1    Alice  Engineering   75000
1   3  Charlie  Engineering   80000
2   5      Eve  Engineering   85000


In [7]:
# ORDER BY
result = conn.execute("""
    SELECT * FROM employees
    ORDER BY salary DESC
""").df()
print("\n💰 Employees by Salary (Highest first):")
print(result)


💰 Employees by Salary (Highest first):
   id     name   department  salary
0   5      Eve  Engineering   85000
1   3  Charlie  Engineering   80000
2   1    Alice  Engineering   75000
3   2      Bob        Sales   65000
4   4    David    Marketing   60000


In [8]:
# Aggregations
result = conn.execute("""
    SELECT
        department,
        COUNT(*) as employee_count,
        AVG(salary) as avg_salary,
        MAX(salary) as max_salary
    FROM employees
    GROUP BY department
    ORDER BY avg_salary DESC
""").df()
print("\n📊 Department Statistics:")
print(result)


📊 Department Statistics:
    department  employee_count  avg_salary  max_salary
0  Engineering               3     80000.0       85000
1        Sales               1     65000.0       65000
2    Marketing               1     60000.0       60000


## 3. Loading Data from Files

DuckDB can read files directly without loading into memory first!

In [9]:
# Find RUP dataset path
project_root = Path.cwd().parent.parent.parent
data_path = project_root / 'datasets' / 'rup' / 'RUP-PaketPenyedia-Terumumkan-2025.parquet'

if data_path.exists():
    print(f"✅ Found dataset: {data_path}")

    # Read Parquet file directly with SQL!
    query = f"""
        SELECT * FROM '{data_path}' LIMIT 10
    """

    df = conn.execute(query).df()
    print("\n📊 RUP Data (first 10 rows):")
    print(df.head())

    # Get row count
    count_query = f"SELECT COUNT(*) as total FROM '{data_path}'"
    total = conn.execute(count_query).fetchone()[0]
    print(f"\n📈 Total records in RUP dataset: {total:,}")

    # Get column info
    info_query = f"DESCRIBE SELECT * FROM '{data_path}'"
    columns_info = conn.execute(info_query).df()
    print("\n📋 Dataset Columns:")
    print(columns_info)

else:
    print(f"⚠️ Dataset not found at: {data_path}")
    print("Creating sample data instead...")

    # Create sample data
    sample_data = pd.DataFrame({
        'nama_paket': ['Paket A', 'Paket B', 'Paket C'],
        'pagu': [1000000000, 2000000000, 1500000000],
        'metode_pengadaan': ['Tender', 'Tender', 'Penunjukan Langsung']
    })

    conn.register('rup_sample', sample_data)
    result = conn.execute("SELECT * FROM rup_sample").df()
    print("\n📊 Sample RUP Data:")
    print(result)

✅ Found dataset: /home/rizko/coding/python/project/data-science/datasets/rup/RUP-PaketPenyedia-Terumumkan-2025.parquet

📊 RUP Data (first 10 rows):
   tahun_anggaran kd_klpd                  nama_klpd jenis_klpd  kd_satker  \
0            2025    D197  Provinsi Kalimantan Barat   PROVINSI     264455   
1            2025    D197  Provinsi Kalimantan Barat   PROVINSI     264456   
2            2025    D197  Provinsi Kalimantan Barat   PROVINSI     264456   
3            2025    D197  Provinsi Kalimantan Barat   PROVINSI     264456   
4            2025    D197  Provinsi Kalimantan Barat   PROVINSI     264456   

            kd_satker_str                                        nama_satker  \
0  1.02.0.00.0.00.03.0000         RUMAH SAKIT JIWA PROVINSI KALIMANTAN BARAT   
1  1.03.0.00.0.00.01.0000  DINAS PEKERJAAN UMUM DAN PENATAAN RUANG PROVIN...   
2  1.03.0.00.0.00.01.0000  DINAS PEKERJAAN UMUM DAN PENATAAN RUANG PROVIN...   
3  1.03.0.00.0.00.01.0000  DINAS PEKERJAAN UMUM DAN PENATAAN RU

## 4. Performance Demonstration

In [10]:
# Create larger dataset for performance test
print("\n⏱️ Performance Test...")

# Generate sample data
np.random.seed(42)
n_rows = 100000

large_df = pd.DataFrame({
    'id': range(n_rows),
    'value': np.random.randint(0, 1000, n_rows),
    'category': np.random.choice(['A', 'B', 'C', 'D'], n_rows)
})

print(f"Created dataset with {n_rows:,} rows")


⏱️ Performance Test...
Created dataset with 100,000 rows


In [11]:
# Method 1: Pandas aggregation
import time

start = time.time()
pandas_result = large_df.groupby('category')['value'].agg(['sum', 'mean', 'count'])
pandas_time = time.time() - start

print(f"\n🐼 Pandas aggregation: {pandas_time:.4f} seconds")
print(pandas_result)


🐼 Pandas aggregation: 0.0098 seconds
               sum        mean  count
category                             
A         12554698  498.083710  25206
B         12560836  502.654608  24989
C         12493046  500.964231  24938
D         12374120  497.612096  24867


In [12]:
# Method 2: DuckDB aggregation
conn.register('large_data', large_df)

start = time.time()
duckdb_result = conn.execute("""
    SELECT
        category,
        SUM(value) as sum,
        AVG(value) as mean,
        COUNT(*) as count
    FROM large_data
    GROUP BY category
""").df()
duckdb_time = time.time() - start

print(f"\n🦆 DuckDB aggregation: {duckdb_time:.4f} seconds")
print(duckdb_result)


🦆 DuckDB aggregation: 0.0090 seconds
  category         sum        mean  count
0        A  12554698.0  498.083710  25206
1        B  12560836.0  502.654608  24989
2        C  12493046.0  500.964231  24938
3        D  12374120.0  497.612096  24867


In [13]:
# Comparison
speedup = pandas_time / duckdb_time
print(f"\n⚡ DuckDB is {speedup:.2f}x faster for this query!")


⚡ DuckDB is 1.10x faster for this query!


## 5. Export Results

In [14]:
# Export to CSV
conn.execute("""
    COPY (
        SELECT * FROM employees
    ) TO 'employees_export.csv' (HEADER, DELIMITER ',')
""")
print("✅ Exported to employees_export.csv")

✅ Exported to employees_export.csv


In [15]:
# Export to Parquet
conn.execute("""
    COPY (
        SELECT * FROM employees
    ) TO 'employees_export.parquet' (FORMAT PARQUET)
""")
print("✅ Exported to employees_export.parquet")

✅ Exported to employees_export.parquet


## 📚 Key Takeaways

1. DuckDB is fast for analytical queries
2. Can read files directly without loading to memory
3. Standard SQL syntax
4. Easy Python integration
5. Perfect for local data analysis

**Next:** Advanced SQL queries and Pandas integration

In [16]:
# Clean up
conn.close()
print("\n✅ Connection closed")


✅ Connection closed


In [17]:
# Summary
print("""
✅ Session 2.1 Complete!

What we learned:
- Creating DuckDB connection
- Basic SQL operations (SELECT, WHERE, ORDER BY)
- Aggregations (COUNT, AVG, MAX, SUM)
- Reading files directly (Parquet, CSV)
- Performance comparison with Pandas
- Export results

Next: Advanced SQL queries and Pandas integration
""")


✅ Session 2.1 Complete!

What we learned:
- Creating DuckDB connection
- Basic SQL operations (SELECT, WHERE, ORDER BY)
- Aggregations (COUNT, AVG, MAX, SUM)
- Reading files directly (Parquet, CSV)
- Performance comparison with Pandas
- Export results

Next: Advanced SQL queries and Pandas integration

